In [145]:
from typing import List, Tuple
import numpy as np

class Simplex:
    """
    The Simplex class implements the Simplex method for solving linear programming problems.
    """
    def __init__(
        self, C: List[float], A: List[float], b: List[float], accuracy: float
    ) -> None:
        """
        Initializes the Simplex method with the following inputs:
        C: Coefficients of the objective function.
        A: Coefficients of the inequality constraints.
        b: Right-hand side values of the inequality constraints.
        accuracy: Precision for detecting optimality (helps handle floating-point errors).
        """
        self.C_coef = np.array(C) # Objective function coefficients
        self.A_coef = np.array(A) # Coefficients of the constraints
        self.b_coef = np.array(b) # Right-hand side values
        self.accuracy = accuracy  # Desired accuracy
        self.table = None         # Simplex table
        self.optimised = False    # Indicates whether the solution is optimized
        self.solvable = True      # Indicates whether the problem is solvable

    def check_infeasibility(self) -> bool:
        """
        If any value in the right-hand side vector b is negative
        and the corresponding row in the matrix A has no positive coefficients,
        the problem is infeasible.
        """
        for i in range(len(self.b_coef)):
            if self.b_coef[i] < 0 and all(self.A_coef[i][j] <= 0 for j in range(len(self.A_coef[i]))):
                return True
        return False
            
    def check_unboudedness(self, ratios: np.ndarray) -> bool:
        """
        If the objective function can grow indefinitely in the direction
        of the feasible region, then the problem is unbounded.
        """
        if np.all(np.isinf(ratios)):
            print("The method is not applicable!")
            self.solvable = False
            return True
        return False

    def fill_initial_table(self) -> None:
        """
        Initializes the Simplex table by combining the constraint matrix A, 
        the identity matrix (for slack variables), and the right-hand side vector b.
        Also appends the objective function row with negative coefficients of C.
        """
        self.table = np.hstack(
            (
                self.A_coef,                        # Coefficients of the constraints
                np.eye(self.A_coef.shape[0]),       # Identity matrix for slack variables
                np.reshape(self.b_coef, (-1, 1)),   # Right-hand side vector b
            )
        )
        # Objective function row (negative coefficients of C)
        func = np.hstack((-self.C_coef, np.zeros(self.A_coef.shape[0] + 1)))
        # Add the objective function row at the bottom
        self.table = np.vstack((self.table, func))

    def make_iteration(self) -> None:
        """
        Performs one iteration of the Simplex algorithm:
        1. Finds the pivot column.
        2. Checks for unboundedness.
        3. Performs the pivot operation to transform the table.
        """

        if self.table is None:
            print("Table was not initialized!")
            return
        
        # Find the most negative value in the objective row
        pivot_column = np.argmin(self.table[-1, :-1])

        # Check if the solution is already optimal
        if self.table[-1, :-1][pivot_column] >= -self.accuracy:
            self.optimised = True
            return
        
        # Compute the ratios for the ratio test
        ratios = np.divide(
            self.table[:-1, -1],                            # Right-hand side values (b)
            self.table[:-1, pivot_column],                  # Pivot column values
            out=np.full_like(self.table[:-1, -1], np.inf),  # Fill with inf where division is not valid
            where=self.table[:-1, pivot_column] > 0,        # Only consider positive entries in the pivot column
        )

        if self.check_unboudedness(ratios):
            return

        # Select the pivot row
        pivot_row = np.argmin(ratios)
        # Normalize the pivot row
        self.table[pivot_row] = (
            self.table[pivot_row] / self.table[pivot_row][pivot_column]
        )
        # Make all other elements in the pivot column zero
        for row in range(self.table.shape[0]):
            if row != pivot_row:
                self.table[row] = (
                    self.table[row]
                    - self.table[row][pivot_column] * self.table[pivot_row]
                )

    def get_solution(self) -> Tuple[List[float], float]:
        """
        Returns the decision variables and the optimized objective function value if the solution exists.
        """

        # Check if the problem is infeasible
        if self.check_infeasibility():
            print(f"The method is not applicable!")
            self.solvable = False
       
        # Perform iterations while the solution is not optimized
        while (not self.optimised) and self.solvable:
            self.make_iteration()
        
        # If the problem is unsolvable, return empty results
        if not self.solvable:
            return [], None
     
        # Initialize solution array (size of decision variables + slack variables)
        solution = np.zeros(
            self.C_coef.shape[0] + self.A_coef.shape[0]
        )

        for row in range(self.A_coef.shape[0]):
            # Find the column index in this row where the value is 1
            for col in range(self.C_coef.shape[0] + self.A_coef.shape[0]):
                # Check if this column is a basic variable
                if self.table[row, col] == 1 and np.sum(self.table[:, col]) == 1:
                    # This is a basic variable column
                    solution[col] = self.table[row, -1]
                    break  # Move to the next row

        # Extract decision variables from the solution
        decision_vars = solution[: self.C_coef.shape[0]]

        # Round to 10 decimal places
        decision_vars = [round(var, 10) for var in decision_vars]
        max_value = round(self.table[-1, -1], 10)

        return decision_vars, max_value

The input contains:  
• A vector of coefficients of objective function - C.  
• A matrix of coefficients of constraint function - A.  
• A vector of right-hand side numbers - b.  
• The approximation accuracy.  

The output contains:  
• The string ”The method is not applicable!”  
or  
• A vector of decision variables - x.  
• Maximum (minimum) value of the objective function.

In [133]:
def get_answer(C: List[float], A: List[float], b:List[float], accuracy: float):
    simplex = Simplex(C, A, b, accuracy)
    simplex.fill_initial_table()
    answer, max_value = simplex.get_solution()
    if not len(answer):
        return
    print("Decision variables:")
    for i in range(len(answer)):
        print(f"x{i + 1} = {answer[i]}")
    print(f"Optimized objective function's value: {max_value}")

### Test #1: correct

In [134]:
# https://www.cuemath.com/algebra/linear-programming/
# When x1 = 4 and x2 = 8 then value of Z = 400
# x2 != 0

C = [40, 30]
A = [[1, 1], [2, 1]]
b = [12, 16]
accuracy = 0.1

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 4.0
x2 = 8.0
Optimized objective function's value: 400.0


### Test #2: correct

In [135]:
# 33 is the maximum value of Z and it occurs at C. Thus, the solution is x = 4 and y = 5.

C = [2, 5]
A = [[1, 4], [3, 1], [1, 1]]
b = [24, 21, 9]
accuracy = 0.5

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 4.0
x2 = 5.0
Optimized objective function's value: 33.0


### Test #3: correct

In [146]:
# We get the maximum value of Z = 27 at x1 = 0, x2 = 9 x3 = 3

C = [1, 2, 3]
A = [[1, 1, 1], [2, 1, 3]]
b = [12, 18]
accuracy = 0.7

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 0.0
x2 = 9.0
x3 = 3.0
Optimized objective function's value: 27.0


### Test #4: correct

In [144]:
# When x1 = 0 and x2 = 8 and x3 = 20 then value of Z = 400

C = [9, 10, 16]
A = [[18, 15, 12], [6, 4, 8], [5, 3, 3]]
b = [360, 192, 180]
accuracy = 0.00001

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 0.0
x2 = 8.0
x3 = 20.0
Optimized objective function's value: 400.0


### Test #5: correct

In [138]:
# When x1 = 0 and x2 = 225.0 and x3 = 0 and x4 = 150 then value of Z = 1050

C = [6, 2, 2.5, 4]
A = [[5, 1, 0, 2], [4, 2, 2, 1], [1, 0, 2, 1]]
b = [1000, 600, 150]
accuracy = 0.00001

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 0.0
x2 = 225.0
x3 = 0.0
x4 = 150.0
Optimized objective function's value: 1050.0


### Test #6: correct

In [139]:
#https://1cov-edu.ru/linejnoe-programmirovanie/simpleks-metod/primer-net-resheniya/?ysclid=m16ir708gt504218275
# Simplex Method is not applicable

C = [4, 5, 4]
A = [[2, 3, -6], [4, 2, -4], [4, 6, -8]]
b = [240, 200, 160]
accuracy = 0.001

get_answer(C, A, b, accuracy)

The method is not applicable!


### Test #7: correct

In [140]:
# This system is infeasible because no values of 𝑥1​ and x2 ​
# can satisfy both inequalities simultaneously.

C = [-1, -1]
A = [[1, 1], [-1,-1]]
b = [1, -3]
accuracy = 0.001

get_answer(C, A, b, accuracy)

The method is not applicable!


### Test #8: correct

In [141]:
#This problem is unbounded because the objective function can increase indefinitely as x1 increases.

C = [2, 1]
A = [[-1, 1]]
b = [1]
accuracy = 0.001

get_answer(C, A, b, accuracy)

The method is not applicable!
